# Introdução

Segundo o site https://developers.mercadolivre.com.br/pt_br/boas-praticas-para-usar-a-plataforma, é recomendado utilizar a API do Mercado Livre em vez de fazer web crawling, portanto foi o caminho escolhido para coletar informações de produtos.

Para funcionar as chamadas, é preciso criar uma conta em developers.mercadolivre.com.br, criar um aplicativo, e conseguir um token de acesso. Esse token deve ser colocar no arquivo .env na variável "ML_ACCESS_TOKEN". Para criar sua aplicação, siga o processo descrito em https://developers.mercadolivre.com.br/pt_br/crie-uma-aplicacao-no-mercado-livre. Após isso, usar o seguinte link pra obter o token de acesso, siga o vídeo https://www.youtube.com/watch?v=T2Q_CE8vqYM&t=53s, cuja explicação está detalhada em https://developers.mercadolivre.com.br/pt_br/autenticacao-e-autorizacao.

IDs de categorias obtidas no site de documentação da API do Mercado Livre:

In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

load_dotenv()

ML_APP_REDIRECT_URI = os.getenv("ML_APP_REDIRECT_URI")
ML_APP_ID = os.getenv("ML_APP_ID")
# url = f"https://auth.mercadolivre.com.br/authorization?response_type=code&client_id={ML_APP_ID}&redirect_uri={ML_APP_REDIRECT_URI}"
# resp = requests.get(url)

# Forma ideal: Obter token de acesso automaticamente (TO-DO)

In [46]:
# Forma atual: Obter token de acesso via navegador, e salvar no arquivo .env
ML_ACCESS_TOKEN = os.getenv("ML_ACCESS_TOKEN")

SITE_ID = "MLB"

headers = {
    "Authorization": f"Bearer {ML_ACCESS_TOKEN}"
}
url = f"https://api.mercadolibre.com/sites/{SITE_ID}/categories"
resp = requests.get(url, headers=headers)
print(resp.json())

[{'id': 'MLB5672', 'name': 'Acessórios para Veículos'}, {'id': 'MLB271599', 'name': 'Agro'}, {'id': 'MLB1403', 'name': 'Alimentos e Bebidas'}, {'id': 'MLB1071', 'name': 'Animais'}, {'id': 'MLB1367', 'name': 'Antiguidades e Coleções'}, {'id': 'MLB1368', 'name': 'Arte, Papelaria e Armarinho'}, {'id': 'MLB1384', 'name': 'Bebês'}, {'id': 'MLB1246', 'name': 'Beleza e Cuidado Pessoal'}, {'id': 'MLB1132', 'name': 'Brinquedos e Hobbies'}, {'id': 'MLB1430', 'name': 'Calçados, Roupas e Bolsas'}, {'id': 'MLB1039', 'name': 'Câmeras e Acessórios'}, {'id': 'MLB1743', 'name': 'Carros, Motos e Outros'}, {'id': 'MLB1574', 'name': 'Casa, Móveis e Decoração'}, {'id': 'MLB1051', 'name': 'Celulares e Telefones'}, {'id': 'MLB1500', 'name': 'Construção'}, {'id': 'MLB5726', 'name': 'Eletrodomésticos'}, {'id': 'MLB1000', 'name': 'Eletrônicos, Áudio e Vídeo'}, {'id': 'MLB1276', 'name': 'Esportes e Fitness'}, {'id': 'MLB263532', 'name': 'Ferramentas'}, {'id': 'MLB12404', 'name': 'Festas e Lembrancinhas'}, {'id':

In [47]:
cat_df = pd.DataFrame(resp.json())
cat_df.head()

,id,name
0,MLB5672,Acessórios para Veículos
1,MLB271599,Agro
2,MLB1403,Alimentos e Bebidas
3,MLB1071,Animais
4,MLB1367,Antiguidades e Coleções


In [5]:
cat_df.shape

(32, 2)

Temos 32 classes. Como o requisito são 100000 produtos reais, eu vou tentar pegar 3125 (= 100000/32) produtos por categoria. Vou tentar usar a API do Mercado Livre para tal

In [6]:
cat_id = cat_df.iloc[1]["id"]

url = f"https://api.mercadolibre.com/sites/{SITE_ID}/search?category=MLB271599"
res = requests.get(url, headers=headers)
res.json()

{'message': 'forbidden', 'error': 'forbidden', 'status': 403, 'cause': []}

As chamadas GET na rota /sites/MLB/search estão retornando 403 (forbidden), e isso também tem sido reportado por outros desenvolvedores (veja [este artigo no Reddit](https://www.reddit.com/r/devsarg/comments/1jovh3s/quien_de_ustedes_rompi%C3%B3_las_apis_de_mercadolibre/?tl=pt-br)). Portanto, a abordagem vai ser utilizar um scraper para obter os IDs, e depois utilizar a API para obter todas as informações pertinentes àquele produto através da rota /products/search, incluindo descrição e outros detalhes adicionais que não aparecem na página de pesquisa.

In [19]:
import requests
from bs4 import BeautifulSoup
import time, re

BASE_URL = "https://lista.mercadolivre.com.br"

def scrape_product_ids_from_search(search_query, max_products=3125, pause=1.5):
    product_ids = []
    offset = 0
    per_page = 50  # ML mostra 50 produtos por página

    while len(product_ids) < max_products:
        url = f"{BASE_URL}/{search_query}_Desde_{offset+1}_NoIndex_True"
        print(f"Scraping page: {url}")
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

        if response.status_code != 200:
            print(f"Erro ao acessar: {url} (status code: {response.status_code})")
            break

        matches = re.findall(r'"product_id":"(MLB\d+)"', response.text)
        unique_ids = list(set(matches))

        product_ids.extend(unique_ids)
        offset += per_page
        time.sleep(pause)

    return list(set(product_ids)) # Retornar só produtos unicos

products_saude = scrape_product_ids_from_search('saude', max_products=50)

Scraping page: https://lista.mercadolivre.com.br/saude_Desde_1_NoIndex_True
Scraping page: https://lista.mercadolivre.com.br/saude_Desde_51_NoIndex_True


In [9]:
products_saude

['MLB20985074',
 'MLB27368691',
 'MLB36188176',
 'MLB26751804',
 'MLB39310342',
 'MLB40442951',
 'MLB34725109',
 'MLB19550318',
 'MLB24336854',
 'MLB44030086',
 'MLB42973846',
 'MLB24000050',
 'MLB27140042',
 'MLB21030116',
 'MLB21814751',
 'MLB25106068',
 'MLB19764367',
 'MLB43079569',
 'MLB49899899',
 'MLB36780934',
 'MLB35509572',
 'MLB34955576',
 'MLB25928611',
 'MLB28364092',
 'MLB29341204',
 'MLB42598574',
 'MLB19403465',
 'MLB23799398',
 'MLB22823332',
 'MLB28508942',
 'MLB38478946',
 'MLB18930516',
 'MLB43269452',
 'MLB36479661',
 'MLB35810512',
 'MLB43941545',
 'MLB21716129',
 'MLB27064907',
 'MLB48929392',
 'MLB49900617',
 'MLB49869909',
 'MLB21652882',
 'MLB19533113',
 'MLB25708438',
 'MLB19549989',
 'MLB19444422',
 'MLB48929367',
 'MLB43057902',
 'MLB43015849',
 'MLB24853579',
 'MLB19930040',
 'MLB19766958',
 'MLB26666122',
 'MLB46254996',
 'MLB24044627',
 'MLB28876499',
 'MLB19446333',
 'MLB21004742',
 'MLB24803533',
 'MLB40063121',
 'MLB44354494',
 'MLB32372413',
 'MLB217

Pelo que entendi da documentação da API, existem itens e existem produtos. Um produto pode ter mais de um item associado (seria anúncio?). Tentei usar a rota /items?ids=$ITEM_ID por ela aceitar solicitar até 20 itens por chamada (preocupação com rate limit), porém também está dando forbidden (erro 403). Então terei que usar a rota /products/search

In [25]:
def get_item_info(item_id):
    url = f"https://api.mercadolibre.com/items?ids={item_id}"
    res = requests.get(url, headers=headers)
    return res.json()

item_info = get_item_info("MLB4736223674")

item_info

[{'code': 403,
  'body': {'id': 'MLB4736223674',
   'message': 'Access to the requested resource is forbidden',
   'error': 'access_denied',
   'status': 403,
   'cause': None}}]

In [43]:
def get_product_info(product_id):
    url = f"https://api.mercadolibre.com/products/search?status=active&site_id=MLB&product_identifier={product_id}"
    print(url)
    res = requests.get(url, headers=headers)
    return res.json()

prod_info = get_product_info(products_saude[2])

prod_info

https://api.mercadolibre.com/products/search?status=active&site_id=MLB&product_identifier=MLB36188176


{'keywords': 'MLB36188176',
 'domain_id': 'MLB-FILLING_MICROCANNULAS',
 'paging': {'total': 1, 'limit': 10, 'offset': 0},
 'results': [{'id': 'MLB36188176',
   'date_created': '2024-04-24T22:50:48Z',
   'catalog_product_id': 'MLB36188176',
   'pdp_types': [],
   'status': 'active',
   'domain_id': 'MLB-FILLING_MICROCANNULAS',
   'settings': {'listing_strategy': 'open', 'exclusive': False},
   'name': 'Microcânula 22g X 50mm Caixa Com 10 Unidades - Smart Gr',
   'main_features': [],
   'attributes': [{'id': 'BRAND',
     'name': 'Marca',
     'value_id': '12326016',
     'value_name': 'Smart GR'},
    {'id': 'MODEL',
     'name': 'Modelo',
     'value_id': '36909331',
     'value_name': 'Smart Micro Cânula 22g X 50mm'},
    {'id': 'SALE_FORMAT',
     'name': 'Formato de venda',
     'value_id': '1359392',
     'value_name': 'Kit'},
    {'id': 'UNITS_PER_PACK',
     'name': 'Unidades por kit',
     'value_id': '150349',
     'value_name': '10'},
    {'id': 'UNITS_PER_PACKAGE',
     'name

{'keywords': 'a',
 'paging': {'total': 0, 'limit': 10, 'offset': 0},
 'results': [],
 'used_attributes': [],
 'query_type': 'PRODUCT_NAME'}